In [3]:
print('hello unsupervised')

hello unsupervised


In [2]:
!conda env list

# conda environments:
#
base                     /home/operation/miniconda3
ds                    *  /home/operation/miniconda3/envs/ds

